In [1]:
# Variable Declaration before 
Silo_Filllevel_1 = 11.6133
Silo_Filllevel_2 = 11.6133
Silo_Filllevel_3 = 11.6133

Hopper_Filllevel_1 = 4.55
Hopper_Filllevel_2 = 0
Hopper_Filllevel_3 = 0

if Silo_Filllevel_1 >= 1:
    Silo_Empty_1 = 1
else:
    Silo_Empty_1 = 0
            
if Silo_Filllevel_2 >= 1:
    Silo_Empty_2 = 1
else:
    Silo_Empty_2 = 0
          
if Silo_Filllevel_3 >= 1:
    Silo_Empty_3 = 1
else:
    Silo_Empty_3 = 0
    
if Hopper_Filllevel_1 >= 1:
    Hopper_Empty_1 = 1
else:
    Hopper_Empty_1 = 0
            
if Hopper_Filllevel_2 >= 1:
    Hopper_Empty_2 = 1
else:
    Hopper_Empty_2 = 0
            
if Hopper_Filllevel_3 >= 1:
    Hopper_Empty_3 = 1
else:
    Hopper_Empty_3 = 0

VP_Prev_2 = 0
VP_Prev_3 = 0

Rising_2 = 0
Falling_2 = 0
Rising_3 = 0
Falling_3 = 0

Q_Flip_2 = 0
Q_Flip_3 = 0

Q_Flip_Prev_2 = 0
Q_Flip_Prev_3 = 0

Saturator_2 = 0
Saturator_3 = 0

MF_S_H_1 = 0
MF_S_H_2 = 0
MF_S_H_3 = 0

MF_VC_2 = 0
MF_VC_3 = 0

Energy_Conveyor_Sim = 0
Energy_VC_2_Sim = 0
Energy_Vibration_Belt_Sim = 0
Energy_VC_3_Sim = 0
Energy_Rotary_Air_Lock_Sim = 0

Silo_Fill_1_Error = []
Time = []
##########
Demand = 50

VP_2 = 0
VP_3 = 1

VP_Time_2 = 10
VP_Time_3 = 10

Exchanged_Mass_Sim = 0
reward_1 = 0
reward_2 = 0
reward_3 = 0
reward_4 = 0
reward_5 = 0

Total_Reward_1 = 0
Total_Reward_2 = 0
Total_Reward_3 = 0
Total_Reward_4 = 0
Total_Reward_5 = 0

In [2]:
#################### Initial State for an episode ###################
import random
from random import randrange, uniform

#if action_counter == 1:
def initial_state():
    
    global Silo_Filllevel_1, Silo_Filllevel_2, Silo_Filllevel_3, Hopper_Filllevel_1, Hopper_Filllevel_2, Hopper_Filllevel_3
    global Silo_Empty_1, Hopper_Empty_1, Q_Flip_2, Q_Flip_3, Q_Flip_Prev_2, Q_Flip_Prev_3, Saturator_2, Saturator_3
    global VP_Prev_2, VP_Prev_3
    
    Silo_Filllevel_1 = uniform(0,17.42)
    Silo_Filllevel_2 = uniform(0,17.42)
    Silo_Filllevel_3 = uniform(0,17.42)
    
    Hopper_Filllevel_1 = uniform(0,10)
    Hopper_Filllevel_2 = uniform(0,10)
    Hopper_Filllevel_3 = uniform(0,10)
    
    if Silo_Filllevel_1 >= 1:
        Silo_Empty_1 = 1
    else:
        Silo_Empty_1 = 0
            
    if Silo_Filllevel_2 >= 1:
        Silo_Empty_2 = 1
    else:
        Silo_Empty_2 = 0
          
    if Silo_Filllevel_3 >= 1:
        Silo_Empty_3 = 1
    else:
        Silo_Empty_3 = 0
    
    if Hopper_Filllevel_1 >= 1:
        Hopper_Empty_1 = 1
    else:
        Hopper_Empty_1 = 0
            
    if Hopper_Filllevel_2 >= 1:
        Hopper_Empty_2 = 1
    else:
        Hopper_Empty_2 = 0
            
    if Hopper_Filllevel_3 >= 1:
        Hopper_Empty_3 = 1
    else:
        Hopper_Empty_3 = 0
    
    VP_Prev_2 = randrange(0,2)
    VP_Prev_3 = randrange(0,2)

    Rising_2 = randrange(0,2)
    Falling_2 = randrange(0,2)
    Rising_3 = randrange(0,2)
    Falling_3 = randrange(0,2)

    Q_Flip_2 = randrange(0,2)
    Q_Flip_3 = randrange(0,2)

    Q_Flip_Prev_2 = randrange(0,2)
    Q_Flip_Prev_3 = randrange(0,2)

In [3]:
######################################### BULK GOOD SYSTEM #####################################
import scipy.integrate as integrate
from scipy.integrate import quad
from math import pow
import matplotlib.pyplot as plt
import numpy as np
import random
from random import randrange, uniform


def environment_bgs():
    
    global Silo_Filllevel_1, Silo_Filllevel_2, Silo_Filllevel_3, Hopper_Filllevel_1, Hopper_Filllevel_2, Hopper_Filllevel_3
    global MF_VC_2, MF_VC_3,Demand
    global Energy_Conveyor_Sim, Energy_VC_2_Sim, Energy_Vibration_Belt_Sim, Energy_VC_3_Sim, Energy_Rotary_Air_Lock_Sim
    global VP_2, VP_3, VP_Time_2, VP_Time_3, MF_S_H_1, MF_S_H_2, MF_S_H_3
    global Silo_Empty_1, Hopper_Empty_1, Q_Flip_2, Q_Flip_3, Q_Flip_Prev_2, Q_Flip_Prev_3, Saturator_2, Saturator_3
    global VP_Prev_2, VP_Prev_3, Exchanged_Mass_Sim, reward_1, reward_2, reward_3, reached
    global Total_Reward_1, Total_Reward_2, Total_Reward_3, Demand
    
    bigloop = 0
    t = 0
    S1 = []     # Silo_1
    S2 = []     # Silo_2
    S3 = []     # Silo_3
    H1 = []     # Hopper_1
    H2 = []     # Hopper_2
    H3 = []     # Hopper_3
    Cb = []     # Cb = Conveyor_belt
    Vb = []     # Vb = Vibration_belt
    Ds = []     # Ds = Dosing_Speed
    while bigloop <=1000:
        
        ##################### Station 1 #####################

        #### Substation 1 - Conveyor Belt - RPM to Mass flow ####


        # Massflow Silo to Hopper

        # Silo Empty
        Silo_Filllevel_1 = uniform(0,18)
        
        if Silo_Filllevel_1 >= 1:
            Silo_Empty_1 = 1
        else:
            Silo_Empty_1 = 0
        #################### 
         # State of silo1
        ###################
        print("Silo_Filllevel_1=",Silo_Filllevel_1)

        if Silo_Filllevel_1 >= 17.45:
            silo_full_1 = 1
        elif 1<Silo_Filllevel_1<17.45:
            silo_full_1 = 1
        else:
            silo_full_1 = 0
        print("silo_full_1=",silo_full_1)    
        if Silo_Filllevel_1 <= 1:
            silo_empty_1 = 1
        else:
            silo_empty_1 = 0
        print("silo_empty_1=",silo_empty_1)
        ####################
        # State of hopper_1
        ####################
        Hopper_Filllevel_1 = uniform(0,10)
        print("Hopper_Filllevel_1",Hopper_Filllevel_1)

        if Hopper_Filllevel_1 >= 9.45:
            hopper_full_1 = 1
        elif 1<Hopper_Filllevel_1<9.45:
            hopper_full_1 = 1
        else:
            hopper_full_1 = 0
        print("hopper_full_1=",hopper_full_1)    
        if Hopper_Filllevel_1 <= 1:
            hopper_empty_1 = 1
        else:
            hopper_empty_1 = 0
        print("hopper_empty_1=",hopper_empty_1)
        
        # Control policies for conveyor
        if hopper_full_1 == 1:
            if silo_full_1 == 1:
                Conveyor_Motor_Speed = 0
                print("Conveyor_Motor_Speed=",Conveyor_Motor_Speed)

        if hopper_empty_1 == 1:
            if silo_full_1 == 1:
                Conveyor_Motor_Speed = 1800
                print("Conveyor_Motor_Speed=",Conveyor_Motor_Speed)
        #####################################

        MF_S_H_1 = Conveyor_Motor_Speed * 0.01 / 60 * Silo_Empty_1

        cms = Conveyor_Motor_Speed

        # Energy - Conveyor Belt

        f_cms = lambda x: Conveyor_Motor_Speed*(-81.0493 + 0.5558316*cms - 0.0008993839*cms**2 + 6.578498e-7*cms**3 - 2.197611e-10*cms**4 + 2.721772e-14*cms**5)/(60*1000)
        energy,err = integrate.quad(f_cms,t,t+0.1)

        Energy_Conveyor_Sim = Energy_Conveyor_Sim +  energy

        #### Substation 2 - Silo ####

        # Silo Fill level

        f_sfl_1 = lambda x: MF_S_H_1 * (-1)
        fill,err = integrate.quad(f_sfl_1,t,t+0.1)

        #torch.save(Silo_Fill_1_Error, 'Silo_Fill_1_Error.pt')

        Silo_Filllevel_1 = Silo_Filllevel_1 + fill
        #Mass_Silo_1 = Mass_Silo_1 + fill
        
        #Silo_Fill_1_Error.append(Silo_Filllevel_1)

        if Silo_Filllevel_1 >= 17.42:
            Silo_Filllevel_1 = 17.42
        elif Silo_Filllevel_1 <= 0:
            Silo_Filllevel_1 = 0
            


        # Silo Empty

        if Silo_Filllevel_1 >= 1:
            Silo_Empty_1 = 1
        else:
            Silo_Empty_1 = 0

        #### Substation 3 - Hopper ####

        # Hopper Fill level
        f_hfl_1 = lambda x: (MF_S_H_1 - MF_VC_2)

        fill, err = integrate.quad(f_hfl_1, t, t+0.1)
        Hopper_Filllevel_1 = Hopper_Filllevel_1 + fill

        if Hopper_Filllevel_1 >= 17.42:
            Hopper_Filllevel_1 = 17.42
        elif Hopper_Filllevel_1 <= 0:
            Hopper_Filllevel_1 = 0

        # Hopper Empty

        if Hopper_Filllevel_1 >= 1:
            Hopper_Empty_1 = 1
        else:
            Hopper_Empty_1 = 0
            
        

       

        ##################### Station 2 ##################### 

        #### Substation 1 - Vacuum Pump ####

        # Massflow - Previous Hopper to Silo

        if t >= VP_Time_2:
            VP_2 = 0
        else:
            VP_2 = 1
        
        if VP_2 - VP_Prev_2 > 0:              # To know if the quantity of material is rising or falling in the VP
            Rising_2 = 1
            Falling_2 = 0     
        elif VP_2 - VP_Prev_2 <0:             #
            Rising_2 = 0 
            Falling_2 = 1     #
        else:                                  #
            Rising_2 = 0 
            Falling_2 = 0     #

        VP_Prev_2 = VP_2

        if Rising_2 == 0 and Falling_2 == 0:    # SR Flip-Flop equivalent
            Q_Flip_2 = Q_Flip_Prev_2
        elif Rising_2 == 0 and Falling_2 == 1:
            Q_Flip_2 = 1
        elif Rising_2 == 1 and Falling_2 == 0:
            Q_Flip_2 = 0
        elif Rising_2 == 1 and Falling_2 == 1:
            Q_Flip_2 = 0

        Q_Flip_Prev_2 = Q_Flip_2

        if Q_Flip_2 == 0:
            MF_VC_2 = 0
        elif Q_Flip_2 == 0:
            Saturator_2 = t - 0.567

        if Saturator_2 >= 4.575:
            Saturator_2 = 4.575
        elif Saturator_2 <= 0:
            Saturator_2 = 0
         ##################  
         # State of silo2
        ##################
        Silo_Filllevel_2 = uniform(0,18)
        print("Silo_Filllevel_2=",Silo_Filllevel_2)
        if Silo_Filllevel_2 >= 17.42:
            silo_full_2 = 1
        elif 1<Silo_Filllevel_2<17.42:
            silo_full_2 = 1
        else:
            silo_full_2 = 0
        print("silo_full_2=",silo_full_2)

        if Silo_Filllevel_2 <= 1:
            silo_empty_2 = 1
        else:
            silo_empty_2 = 0
        print("silo_empty_2=",silo_empty_2)
        ##################
        #State of hopper2
        ##################
        Hopper_Filllevel_2 = uniform(0,10)
        print("Hopper_Filllevel_2",Hopper_Filllevel_2)

        if Hopper_Filllevel_2 >= 9.45:
            hopper_full_2 = 1
        elif 1<Hopper_Filllevel_2<9.45:
            hopper_full_2 = 1
        else:
            hopper_full_2 = 0
        print("hopper_full_2=",hopper_full_2)    
        if Hopper_Filllevel_2 <= 1:
            hopper_empty_2 = 1
        else:
            hopper_empty_2 = 0
        print("hopper_empty_2=",hopper_empty_2)
        
        #########################################
        
         # Control policies for vaccumpump
        if silo_full_2 == 1:
            if hopper_full_1==0 and hopper_empty_1==1:
                saturator_2 = 0
            elif hopper_full_1==1 and hopper_empty_1==0:
                saturator_2 = 0
        elif silo_empty_2 == 1:
            if hopper_full_1==1 and hopper_empty_1==0:
                saturator_2 = 4.575
            elif hopper_full_1==0 and hopper_empty_1==1:
                saturator_2 = 0
        print("saturator_2=",saturator_2)
        ##########################################

        MF_VC_2 = ((0.0664 * Saturator_2) + 0.464) * (Hopper_Empty_1)

        # Energy - Vacuum Pump

        f_vp_2 = lambda x: (VP_2 * 0.305)

        energy,err = integrate.quad(f_vp_2, t, t+0.1)

        Energy_VC_2_Sim = Energy_VC_2_Sim + energy
        ###########################################
        # Control policies for vibration belt
        if hopper_full_2 == 1:
            if silo_full_2 == 1:
                Vibration_Belt_Start = 0
                print("Vibration_Belt_Start=", Vibration_Belt_Start)

        if hopper_empty_2 == 1:
            if silo_full_2 == 1:
                Vibration_Belt_Start = 1
                print("Vibration_Belt_Start=",Vibration_Belt_Start)
        #############################################

        #### Substation 2- Vibration Belt ####

        # Massflow Silo to Hopper

        # Silo Empty

        if Silo_Filllevel_2 >= 1:
            Silo_Empty_2 = 1
        else:
            Silo_Empty_2 = 0

        MF_S_H_2 = Vibration_Belt_Start * 0.4 * Silo_Empty_2

        # Energy Vibration Belt

        f_env = lambda x: (26.9 * Vibration_Belt_Start / 1000)
        energy, err = integrate.quad(f_env,t,t+0.1)

        Energy_Vibration_Belt_Sim = Energy_Vibration_Belt_Sim + energy

        #### Substation 3 - Silo ####

        # Silo Fill level

        f_sfl_2 = lambda x: (MF_VC_2 - MF_S_H_2)
        fill,err = integrate.quad(f_sfl_2,t,t+0.1)

        Silo_Filllevel_2  = Silo_Filllevel_2 + fill
        #Mass_Silo_2 = Mass_Silo_2 + fill

        if Silo_Filllevel_2 >= 17.42:
            Silo_Filllevel_2 = 17.42
        elif Silo_Filllevel_2 <= 0:
            Silo_Filllevel_2 = 0


        # Silo Empty

        if Silo_Filllevel_2 >= 1:
            Silo_Empty_2 = 1
        else:
            Silo_Empty_2 = 0

        #### Substation 3 - Hopper ####

        # Hopper Fill level

        f_hfl_2 = lambda x: (MF_S_H_2 - MF_VC_3)

        fill,err = integrate.quad(f_hfl_2,t,t+0.1)

        Hopper_Filllevel_2 = Hopper_Filllevel_2 + fill

        if Hopper_Filllevel_2 >= 17.42:
            Hopper_Filllevel_2 = 17.42
        elif Hopper_Filllevel_2 <= 0:
            Hopper_Filllevel_2 = 0

        # Hopper Empty

        if Hopper_Filllevel_2 >= 1:
            Hopper_Empty_2 = 1
        else:
            Hopper_Empty_2 = 0
            
           
       ##################### Station 3 ##################### 

        #### Substation 1 - Vacuum Pump ####

        if t >= VP_Time_3:
            VP_3 = 0
        else: 
            VP_3 = 0

        if VP_3 - VP_Prev_3 > 0:              # To know if the quantity of material is rising or falling in the VP
            Rising_3 = 1 
            Falling_3 = 0      #
        elif VP_3 - VP_Prev_2 <0:             #
            Rising_3 = 0 
            Falling_3 = 1      #
        elif VP_3 - VP_Prev_3 == 0:            #
            Rising_3 = 0 
            Falling_3 = 0      #

        VP_Prev_3 = VP_3

        if Rising_3 == 0 and Falling_3 == 0:    # SR Flip-Flop equivalent
            Q_Flip_3 = Q_Flip_Prev_3
        elif Rising_3 == 0 and Falling_3 == 1:
            Q_Flip_3 = 1
        elif Rising_3 == 1 and Falling_3 == 0:
            Q_Flip_3 = 0
        elif Rising_3 == 1 and Falling_3 == 1:
            Q_Flip_3 = 0

        Q_Flip_Prev_3 = Q_Flip_3

        if Q_Flip_3 == 0:
            MF_VC_3 = 0
        elif Q_Flip_3 == 0:
            Saturator_3 = t - 0.979

        if Saturator_3 >= 9.5:
            Saturator_3 = 9.5
        elif Saturator_3 <= 0:
            Saturator_3 = 0
        
        ###################
         # State of silo3
        ##################
        Silo_Filllevel_3 = uniform(0,18)
        print("Silo_Filllevel_3=",Silo_Filllevel_3)
        if Silo_Filllevel_3 >= 17.42:
            silo_full_3 = 1
        elif 1<Silo_Filllevel_3<17.42:
            silo_full_3 = 1
        else:
            silo_full_3 = 0
        print("silo_full_3=",silo_full_3)

        if Silo_Filllevel_3 <= 1:
            silo_empty_3 = 1
        else:
            silo_empty_3 = 0
        print("silo_empty_3=",silo_empty_3) 
        ###################
        #State of hopper3
        ###################
        Hopper_Filllevel_3 = uniform(0,10)
        print("Hopper_Filllevel_3",Hopper_Filllevel_3)

        if Hopper_Filllevel_3 >= 9.45:
            hopper_full_3 = 1
        elif 1<Hopper_Filllevel_3<9.45:
            hopper_full_3 = 1
        else:
            hopper_full_3 = 0
        print("hopper_full_3=",hopper_full_3)    
        if Hopper_Filllevel_3 <= 1:
            hopper_empty_3 = 1
        else:
            hopper_empty_3 = 0
        print("hopper_empty_3=",hopper_empty_3)
        ##########################################
        if silo_full_3 == 1:
            if hopper_full_2==0 and hopper_empty_2==1:
                saturator_3 = 0
            elif hopper_full_2==1 and hopper_empty_2==0:
                saturator_3 = 0
        elif silo_empty_3 == 1:
            if hopper_full_2==1 and hopper_empty_2==0:
                saturator_3 = 9.5
            elif hopper_full_2==0 and hopper_empty_2==1:
                saturator_3 = 0
        print("saturator_3=",saturator_3)
        ############################################
        MF_VC_3 = ((0.0192 * Saturator_3) + 0.3535) * (Hopper_Empty_2)

        # Energy - Vacuum Pump

        f_vp_3 = lambda x: (VP_3 * 0.456)

        energy,err = integrate.quad(f_vp_3,t,t+0.1)

        Energy_VC_3_Sim = Energy_VC_3_Sim + energy
        ##############################
        # Control policies for Dosing speed  # we considered here silo1 as hopper to silo
        if hopper_full_3 == 1:                 # because of cyclic transfer of material between stations
            if silo_full_3 == 1:
                Dosing_Speed = 0
                print("Dosing_Speed=", Dosing_Speed)

        if hopper_empty_3 == 1:
            if silo_full_3 == 1:
                Dosing_Speed = 600
                print("Dosing_Speed=",Dosing_Speed)
        #################################

        #### Substation 2 - Rotary Air lock ####

        # Massflow Silo to Hopper

        # Silo Empty

        if Silo_Filllevel_3 >= 1:
            Silo_Empty_3 = 1
        else:
            Silo_Empty_3 = 0

        MF_S_H_3 = (Dosing_Speed) * (Silo_Empty_3) * 0.01249 / 60

        # Energy Rotary Air Lock

        f_eral = lambda x: (Dosing_Speed * 370 / (1500 * 1000))

        energy,err = integrate.quad(f_eral,t,t+0.1)

        Energy_Rotary_Air_Lock_Sim = Energy_Rotary_Air_Lock_Sim + energy

        #### Substation 3 - Silo ####

        # Silo Fill level

        f_sfl_3 = lambda x: (MF_VC_3 - MF_S_H_3)
        fill,err = integrate.quad(f_sfl_3,t,t+0.1)

        Silo_Filllevel_3  = Silo_Filllevel_3 + fill
        #Mass_Silo_3 = Mass_Silo_3 + fill

        if Silo_Filllevel_3 >= 17.42:
            Silo_Filllevel_3 = 17.42
        elif Silo_Filllevel_3 <= 0:
            Silo_Filllevel_3 = 0

        # Silo Empty

        if Silo_Filllevel_3 >= 1:
            Silo_Empty_3 = 1
        else:
            Silo_Empty_3 = 0

        #### Substation 3 - Hopper ####

        # Hopper Fill level

        f_hfl_3 = lambda x: (MF_S_H_1 - Demand)

        fill,err = integrate.quad(f_hfl_3,t,t+0.1)
        
        Hopper_Filllevel_3 = Hopper_Filllevel_3 + fill

        if Hopper_Filllevel_3 >= 17.42:
            Hopper_Filllevel_3 = 17.42
        elif Hopper_Filllevel_3 <= 0:
            Hopper_Filllevel_3 = 0
    
        f_demand = lambda x: Demand                      # Overall Mass transported by the system
        dem,err = integrate.quad(f_demand,t,t+0.1)       #
        Exchanged_Mass_Sim = Exchanged_Mass_Sim + dem    #

        # Hopper Empty

        if Hopper_Filllevel_3 >= 1:
            Hopper_Empty_3 = 1
        else:
            Hopper_Empty_3 = 0
       
       
        ##########################################################

        t = t + 0.1            # Next time step 
        ##############
        #Collection of data
        ###############
        S1.append(Silo_Filllevel_1)
        S2.append(Silo_Filllevel_2)
        S3.append(Silo_Filllevel_3)
        H1.append(Hopper_Filllevel_1)
        H2.append(Hopper_Filllevel_2)
        H3.append(Hopper_Filllevel_3)
        Cb.append(Conveyor_Motor_Speed)
        Vb.append(Vibration_Belt_Start)
        Ds.append(Dosing_Speed)
        

        bigloop = bigloop + 1
        print('bigloop=',bigloop)
        ################################################################################################################   
    Total_Energy_Sim = sum([Energy_Conveyor_Sim, Energy_VC_2_Sim, Energy_Vibration_Belt_Sim, Energy_VC_3_Sim, Energy_Rotary_Air_Lock_Sim])
    ##########
    #Convertion in to Array
    ##########
    y1 = np.asarray(S1)
    y2 = np.asarray(S2)
    y3 = np.asarray(S3)
    y4 = np.asarray(H1)
    y5 = np.asarray(H2)
    y6 = np.asarray(H3)
    y7 = np.asarray(Cb)
    y8 = np.asarray(Vb)
    y9 = np.asarray(Ds)
    ###########
    #Plotting
    ###########
    
    %matplotlib qt
    plt.figure()
    
    plt.plot(y1,label='Silo_1')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    plt.figure()
    plt.plot(y2,label='Silo_2')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    plt.figure()
    plt.plot(y3,label='Silo_3')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    
    plt.figure()
    plt.plot(y4,label='Hopper_1')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    plt.figure()
    plt.plot(y5,label='Hopper_2')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    plt.figure()
    plt.plot(y6,label='Hopper_3')
    plt.xlabel('Cycles')
    plt.ylabel('State')
    plt.grid(True)
    
    plt.figure()
    plt.plot(y7,label='Conveyor_belt')
    plt.xlabel('Cycles')
    plt.ylabel('Speed')
    plt.grid(True)
    
    plt.figure()
    plt.plot(y8,label='Vibration_Belt_Speed')
    plt.xlabel('Cycles')
    plt.ylabel('Speed')
    plt.grid(True)
    
    plt.figure()
    plt.plot(y9,label='Dosing_Speed')
    plt.xlabel('Cycles')
    plt.ylabel('Speed')
    plt.grid(True)
    
    return  Silo_Filllevel_1, Silo_Filllevel_2, Silo_Filllevel_3, Hopper_Filllevel_1, Hopper_Filllevel_2, Hopper_Filllevel_3, Energy_Conveyor_Sim, Energy_VC_2_Sim, Energy_Vibration_Belt_Sim, Energy_VC_3_Sim, Energy_Rotary_Air_Lock_Sim, Total_Energy_Sim, VP_2, VP_3, VP_Time_2, VP_Time_3, MF_VC_2, MF_VC_3, MF_S_H_1, MF_S_H_2, MF_S_H_3, Silo_Empty_1, Hopper_Empty_1, Q_Flip_2, Q_Flip_3, Q_Flip_Prev_2, Q_Flip_Prev_3, Saturator_2, Saturator_3, VP_Prev_2, VP_Prev_3, Exchanged_Mass_Sim,Demand, Total_Energy_Sim 

# 0 Silo_Filllevel_1      # 7 Energy_VC_2_Sim              #14 VP_Time_2    #21 Silo_Empty_1      #28 Saturator_3          #35 reward_3
# 1 Silo_Filllevel_2      # 8 Energy_Vibration_Belt_Sim    #15 VP_Time_3    #22 Hopper_Empty_1    #29 VP_Prev_2            #36 Total_Reward_1
# 2 Silo_Filllevel_3      # 9 Energy_VC_3_Sim              #16 MF_VC_2      #23 Q_Flip_2          #30 VP_Prev_3            #37 Total_Reward_2
# 3 Hopper_Filllevel_1    #10 Energy_Rotary_Air_Lock_Sim   #17 MF_VC_3      #24 Q_Flip_3          #31 Exchanged_Mass_Sim   #38 Total_Reward_1
# 4 Hopper_Filllevel_2    #11 Total_Energy_Sim             #18 MF_S_H_1     #25 Q_Flip_Prev_2     #32 reached              #39 Demand
# 5 Hopper_Filllevel_3    #12 VP_2                         #19 MF_S_H_2     #26 Q_Flip_Prev_3     #33 reward_1             #40 Total_Energy_Sim
# 6 Energy_Conveyor_Sim   #13 VP_3                         #20 MF_S_H_3     #27 Saturator_2       #34 reward_2

In [6]:
new_state = environment_bgs()

Silo_Filllevel_1= 11.420070101209356
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 7.232621965681084
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 15.198836509460845
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 9.35695149245888
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 1.0382286640290532
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 3.261484824082702
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 1
Silo_Filllevel_1= 15.765907795110634
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 1.631541846940523
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 3.2794090588112406
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 3.2868966782572926
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 8.166528396405358
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 9.499166706565877
hopper_full_3= 1
hopper_empt

hopper_empty_3= 1
saturator_3= 0
Dosing_Speed= 600
bigloop= 63
Silo_Filllevel_1= 0.537518557738466
silo_full_1= 0
silo_empty_1= 1
Hopper_Filllevel_1 0.9518028388037991
hopper_full_1= 0
hopper_empty_1= 1
Silo_Filllevel_2= 7.116129402554256
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 3.2323677870587852
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 15.45672632943857
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 3.1970154491740113
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 64
Silo_Filllevel_1= 5.861859537493418
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 9.007731125486064
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 15.005225096711536
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 0.03699289969784303
hopper_full_2= 0
hopper_empty_2= 1
saturator_2= 0
Vibration_Belt_Start= 1
Silo_Filllevel_3= 1.4651052190741267
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 2.18

hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 10.159496726423793
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 6.884441452512236
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 105
Silo_Filllevel_1= 4.2798611751018765
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 4.813286119852536
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 9.9622700924248
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 7.507240546221756
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 9.477058887671724
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 4.09971935902562
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 106
Silo_Filllevel_1= 12.27267051182693
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 1.770068521453373
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 1.4656273197775893
silo_full_2= 1
silo_

hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 156
Silo_Filllevel_1= 2.5656708400672374
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 7.585348482212146
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 9.803089148041122
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 3.7101328520420775
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 9.777173085146016
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 5.4692875818119635
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 157
Silo_Filllevel_1= 0.7735186057007954
silo_full_1= 0
silo_empty_1= 1
Hopper_Filllevel_1 3.587758605415271
hopper_full_1= 1
hopper_empty_1= 0
Silo_Filllevel_2= 3.49596868738544
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 7.774803515547103
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 0.9744916228522837
silo_full_3= 0
silo_empty_3= 1
Hopper_Filllevel_3 8.64873

bigloop= 201
Silo_Filllevel_1= 2.7733288798697786
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 0.5549072842496305
hopper_full_1= 0
hopper_empty_1= 1
Conveyor_Motor_Speed= 1800
Silo_Filllevel_2= 17.64614929568283
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 1.8379854707519228
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 16.061080160443954
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 4.1967772214191985
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 202
Silo_Filllevel_1= 12.268798131378206
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 2.566289389570774
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 17.205383727239354
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 7.465913613160225
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 3.7748530209078694
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 7.7611334879376255
hopper

...

hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 798
Silo_Filllevel_1= 8.027381281499116
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 4.020579634014374
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 11.079606837348534
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 0.27241583838058303
hopper_full_2= 0
hopper_empty_2= 1
saturator_2= 0
Vibration_Belt_Start= 1
Silo_Filllevel_3= 5.914844485232623
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 7.7362870449912755
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 799
Silo_Filllevel_1= 1.9554488633037537
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 8.083625335845174
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 17.88788626898039
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 6.179274708114093
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 9.894313837377647
silo_full_3= 1
silo_empty_3= 0
H

silo_full_2= 0
silo_empty_2= 1
Hopper_Filllevel_2 0.5913552960321722
hopper_full_2= 0
hopper_empty_2= 1
saturator_2= 4.575
Silo_Filllevel_3= 1.7181524119297729
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 2.0716930812759546
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 836
Silo_Filllevel_1= 16.447423996839508
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 2.2784755987422023
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 2.694035741573182
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 8.691701903306544
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 6.976878396913328
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 1.4107396045896858
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 837
Silo_Filllevel_1= 8.496922415577348
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 7.256362789887162
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Si

hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 11.03305734945243
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 0.3355440845185187
hopper_full_3= 0
hopper_empty_3= 1
saturator_3= 0
Dosing_Speed= 600
bigloop= 884
Silo_Filllevel_1= 5.110503738520961
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 2.9471427101994507
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 1.9084164665528176
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 4.791794165977741
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 16.236395906000215
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 2.5408810778877156
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 885
Silo_Filllevel_1= 1.2084185036927229
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 0.697788180286345
hopper_full_1= 0
hopper_empty_1= 1
Conveyor_Motor_Speed= 1800
Silo_Filllevel_2= 4.605045128795453
silo_full

silo_full_3= 0
silo_empty_3= 1
Hopper_Filllevel_3 8.944185462629791
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 9.5
bigloop= 927
Silo_Filllevel_1= 12.58450003770832
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 8.421620781916731
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 6.4077761173125785
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 8.106598884968658
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 1.9215180911364391
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 9.485927979024474
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 928
Silo_Filllevel_1= 12.555943329194308
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 4.906140660278867
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 16.813764518292942
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 2.056158491037756
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 

Vibration_Belt_Start= 0
Silo_Filllevel_3= 1.2296920714444923
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 3.941381398308507
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 972
Silo_Filllevel_1= 15.068092323132687
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 0.9275494276657748
hopper_full_1= 0
hopper_empty_1= 1
Conveyor_Motor_Speed= 1800
Silo_Filllevel_2= 5.896379599286653
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 1.8072383457606023
hopper_full_2= 1
hopper_empty_2= 0
saturator_2= 0
Vibration_Belt_Start= 0
Silo_Filllevel_3= 4.3279628867378825
silo_full_3= 1
silo_empty_3= 0
Hopper_Filllevel_3 2.0986618234935337
hopper_full_3= 1
hopper_empty_3= 0
saturator_3= 0
Dosing_Speed= 0
bigloop= 973
Silo_Filllevel_1= 3.4782976029280364
silo_full_1= 1
silo_empty_1= 0
Hopper_Filllevel_1 5.048679885822858
hopper_full_1= 1
hopper_empty_1= 0
Conveyor_Motor_Speed= 0
Silo_Filllevel_2= 6.350383171927077
silo_full_2= 1
silo_empty_2= 0
Hopper_Filllevel_2 9.62891910